# 🌾 AgriVision Pro Tutorial

## Satellite & Drone Vegetation Analysis

This notebook teaches you how to:
1. Run the AgriVision Pro Streamlit app
2. Authenticate with Google Earth Engine
3. Load satellite imagery
4. Calculate vegetation indices
5. Analyze drone/RGB images
6. Create time series charts

---
## 🚀 Quick Start: Run the App

Run the cell below to start the AgriVision Pro Streamlit app!

In [ ]:
# Run the AgriVision Pro Streamlit App
# This will open the app in a new browser tab

import subprocess
import webbrowser
import time

print("🌾 Starting AgriVision Pro...")
print("The app will open in your browser at http://localhost:8501")

# Start the Streamlit app
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait a bit for the server to start
time.sleep(3)

# Open in browser
webbrowser.open('http://localhost:8501')

print("✅ App started! Check your browser.")
print("\nTo stop the app, run: process.terminate()")

In [ ]:
# Run this cell to STOP the app
try:
    process.terminate()
    print("✅ App stopped.")
except:
    print("App was already stopped or not running.")

---
## Step 1: Install and Import Libraries

First, install the required packages if you haven't already.

In [ ]:
# Install required packages (run this once)
# Uncomment the line below to install
# !pip install streamlit geemap earthengine-api matplotlib numpy pandas pillow

In [ ]:
# Import libraries
import ee                     # Google Earth Engine
import geemap                 # Interactive maps
import numpy as np            # Numerical operations
import matplotlib.pyplot as plt  # Plotting
import datetime               # Date handling

print("✅ Libraries imported successfully!")

---
## Step 2: Authenticate with Google Earth Engine

You need a Google Earth Engine account. If you don't have one, sign up at:
https://signup.earthengine.google.com/

In [ ]:
# Authenticate (only needed once)
# This will open a browser window for you to log in
try:
    ee.Initialize()
    print("✅ Already authenticated!")
except:
    ee.Authenticate()
    ee.Initialize()
    print("✅ Authentication complete!")

---
## Step 3: Define Your Area of Interest

Enter the coordinates of the area you want to analyze.

In [ ]:
# Set your location (change these values)
latitude = 39.0    # Your latitude
longitude = -98.0  # Your longitude
buffer_km = 10     # Size of area in kilometers

# Create a point and buffer it to create an area
point = ee.Geometry.Point([longitude, latitude])
area_of_interest = point.buffer(buffer_km * 1000)  # Convert km to meters

# Create a map to visualize
Map = geemap.Map(center=[latitude, longitude], zoom=10)
Map.addLayer(area_of_interest, {'color': 'blue'}, 'Area of Interest')
Map

---
## Step 4: Load Sentinel-2 Imagery

Sentinel-2 is a European satellite with 10m resolution, perfect for agricultural analysis.

**Default dates:**
- End date = Today
- Start date = 1 month before today

In [ ]:
# Set your date range (end = today, start = 1 month before)
from datetime import date, timedelta

end_date = date.today().strftime('%Y-%m-%d')
start_date = (date.today() - timedelta(days=30)).strftime('%Y-%m-%d')

print(f"Date range: {start_date} to {end_date}")

# Load Sentinel-2 image collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

# Filter the collection
filtered = sentinel2 \
    .filterDate(start_date, end_date) \
    .filterBounds(area_of_interest) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

print(f"Found {filtered.size().getInfo()} images")

In [ ]:
# Create a cloud-free composite (median of all images)

def mask_clouds(image):
    """Remove clouds from Sentinel-2 image using QA60 band"""
    qa = image.select('QA60')
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(
           qa.bitwiseAnd(cirrus_bit).eq(0))
    return image.updateMask(mask)

# Apply cloud mask and create median composite
composite = filtered.map(mask_clouds).median().clip(area_of_interest)

print("✅ Cloud-free composite created!")

In [ ]:
# Display the true color image (RGB)
rgb_params = {
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'min': 0,
    'max': 3000
}

Map = geemap.Map(center=[latitude, longitude], zoom=11)
Map.addLayer(composite, rgb_params, 'True Color')
Map.addLayer(area_of_interest, {'color': 'yellow'}, 'AOI', opacity=0.3)
Map

---
## Step 5: Calculate NDVI (Normalized Difference Vegetation Index)

NDVI is the most common vegetation index.

**Formula:** NDVI = (NIR - Red) / (NIR + Red)

**Interpretation:**
- Values close to 1 = Dense, healthy vegetation
- Values around 0.2-0.5 = Moderate vegetation
- Values close to 0 = Bare soil
- Negative values = Water, snow

In [ ]:
# Calculate NDVI
# In Sentinel-2: B8 = NIR, B4 = Red

ndvi = composite.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Display NDVI map
ndvi_params = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
}

Map = geemap.Map(center=[latitude, longitude], zoom=11)
Map.addLayer(ndvi, ndvi_params, 'NDVI')
Map.add_colorbar(ndvi_params, label='NDVI Value')
Map

---
## Step 6: Calculate Other Vegetation Indices

Let's try some other useful indices.

In [ ]:
# Get band references
nir = composite.select('B8')
red = composite.select('B4')
green = composite.select('B3')
blue = composite.select('B2')
red_edge = composite.select('B5')

# SAVI - Soil Adjusted Vegetation Index
L = 0.5
savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')

# EVI - Enhanced Vegetation Index
evi = nir.subtract(red).multiply(2.5).divide(
    nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)
).rename('EVI')

# NDRE - Normalized Difference Red Edge Index
ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE')

print("✅ All indices calculated!")

In [ ]:
# Compare all indices side by side
Map = geemap.Map(center=[latitude, longitude], zoom=11)

vis = {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}

Map.addLayer(ndvi, vis, 'NDVI')
Map.addLayer(savi, vis, 'SAVI')
Map.addLayer(evi, vis, 'EVI')
Map.addLayer(ndre, vis, 'NDRE')

Map.add_layer_control()
Map

---
## Step 7: RGB-Based Indices (for Drone/Camera Images)

These indices work with just Red, Green, Blue bands - perfect for drone images!

In [ ]:
# Create a sample RGB array (simulating a drone image)
np.random.seed(42)
rows, cols = 200, 300

# Create masks for vegetation and soil areas
vegetation_mask = np.random.random((rows, cols)) > 0.4

# Create RGB channels
sample_image = np.zeros((rows, cols, 3), dtype=np.float32)

# Vegetation areas (green)
sample_image[vegetation_mask, 0] = 50 + np.random.randint(0, 30, vegetation_mask.sum())
sample_image[vegetation_mask, 1] = 120 + np.random.randint(0, 40, vegetation_mask.sum())
sample_image[vegetation_mask, 2] = 40 + np.random.randint(0, 20, vegetation_mask.sum())

# Soil areas (brown)
sample_image[~vegetation_mask, 0] = 140 + np.random.randint(0, 30, (~vegetation_mask).sum())
sample_image[~vegetation_mask, 1] = 100 + np.random.randint(0, 30, (~vegetation_mask).sum())
sample_image[~vegetation_mask, 2] = 60 + np.random.randint(0, 20, (~vegetation_mask).sum())

sample_image = sample_image / 255.0

plt.figure(figsize=(10, 6))
plt.imshow(sample_image)
plt.title('Sample Drone Image (Simulated)')
plt.axis('off')
plt.show()

In [ ]:
# Calculate RGB-based indices
R = sample_image[:, :, 0]
G = sample_image[:, :, 1]
B = sample_image[:, :, 2]

# ExG - Excess Green
exg = 2*G - R - B

# GLI - Green Leaf Index
gli = (2*G - R - B) / np.maximum(2*G + R + B, 0.0001)

# VARI - Visible Atmospherically Resistant Index
vari = np.clip((G - R) / np.maximum(G + R - B, 0.0001), -1, 1)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(sample_image)
axes[0, 0].set_title('Original RGB Image')
axes[0, 0].axis('off')

im1 = axes[0, 1].imshow(exg, cmap='RdYlGn', vmin=-0.5, vmax=0.5)
axes[0, 1].set_title('ExG (Excess Green)')
axes[0, 1].axis('off')
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[1, 0].imshow(gli, cmap='RdYlGn', vmin=-1, vmax=1)
axes[1, 0].set_title('GLI (Green Leaf Index)')
axes[1, 0].axis('off')
plt.colorbar(im2, ax=axes[1, 0])

im3 = axes[1, 1].imshow(vari, cmap='RdYlGn', vmin=-1, vmax=1)
axes[1, 1].set_title('VARI')
axes[1, 1].axis('off')
plt.colorbar(im3, ax=axes[1, 1])

plt.suptitle('RGB-Based Vegetation Indices', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Summary: Which Index Should You Use?

| Your Situation | Recommended Index |
|----------------|-------------------|
| General vegetation health | **NDVI** |
| Sparse vegetation / visible soil | **SAVI** |
| Dense forest / high biomass | **EVI** |
| Chlorophyll / nitrogen content | **NDRE** (Sentinel-2) or **GNDVI** |
| Water stress / drought | **NDMI** |
| Drone imagery (RGB only) | **GLI** or **ExG** |
| Quick simple check | **NGRDI** |

---
## 🎉 Congratulations!

You've learned how to use AgriVision Pro!

✅ Run the Streamlit app from this notebook
✅ Authenticate with Google Earth Engine
✅ Load and process satellite imagery
✅ Calculate vegetation indices (NDVI, SAVI, EVI, NDRE)
✅ Use RGB-only indices for drone images

### Next Steps:
1. Try with your own coordinates
2. Upload your own drone images in the app
3. Use the Compare Images feature
4. Check visitor analytics!